<a href="https://colab.research.google.com/github/BrianKEverett/County-Line/blob/main/Everett_PS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#---------------------------SETUP----------------------------------
#get useful libraries
import time, os, sys, re #basics
import zipfile, json, datetime, string   #string for annotating points in scatter
import numpy as np #basic math
from statistics import * #stats

import matplotlib.pyplot as plt #import pylab as plt #apparently discouraged now:
 #https://stackoverflow.com/questions/11469336/what-is-the-difference-between-pylab-and-pyplot
 #https://www.tutorialspoint.com/matplotlib/matplotlib_pylab_module.htm

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
#s4 = Styler(df4, uuid_len=0, cell_ids=False)

from google.colab import files

#import webbrowser

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter() #this enables spreadsheet view upon calling dataframe (without() )

#many tricks how to extend notebook functionality
#https://coderzcolumn.com/tutorials/python/list-of-useful-magic-commands-in-jupyter-notebook-lab
#will display all output not just last command
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#MAGICS and THEMES/STYLES: important! does affect not just shading/colors, but also fonts, spacing, etc
#(even if you only select default (v not selecting anything) [but does seem to work better if you do make explicit sleections])

###magics: https://ipython.readthedocs.io/en/stable/interactive/magics.html
#most essential setup for vis: it does affect vis! careful!! stick with inline, maybe notebook; others mostly for non-notebook, eg spyder environ
#https://jakevdp.github.io/PythonDataScienceHandbook/04.00-introduction-to-matplotlib.html recomends *inline*!
#show current one:
#%matplotlib
#%matplotlib --list
#interactive plots:
#%matplotlib notebook
#static images of your plot:
%matplotlib inline
#may play with this one and other magics (btw default is probably agg)
#%matplotlib nbagg

###themes/styles: https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html
#https://jakevdp.github.io/PythonDataScienceHandbook/04.11-settings-and-stylesheets.html
#https://matplotlib.org/stable/tutorials/introductory/customizing.html
#here more about art and style than under the hood functionality as with magics, explore and experiment
#many may find 'default' or seaborn ones more pleasing; my fav 'classic' is back from 90s ;)
#plt.style.available #list available styles :) may install more
#plt.style.use('default') # more delicate subtle than classic
plt.style.use('classic')  #  'seaborn-whitegrid' 'seaborn-white' 'seaborn-poster'
# btw: magics v theme/style sequence matters, eg if i specify classic style before inline magic, i wouldnt get grey bounding box im getting

Just setting up

In [2]:
!wget -q -O CD1PS1T2.csv https://github.com/BrianKEverett/County-Line/raw/main/CD1PS1T2.csv

In [5]:
dat=pd.read_csv('CD1PS1T2.csv')

In [6]:
dat.columns

Index(['County', 'Municipality', 'DLGS', 'LD', 'CD', '2020RV', '2020DEMS',
       '2020REPS', '2020VT'],
      dtype='object')

In [7]:
dat

,County,Municipality,DLGS,LD,CD,2020RV,2020DEMS,2020REPS,2020VT
0,Burlington,Maple Shade,319,6,1,13247,38.09,17.59,71.59
1,Burlington,Palmyra,327,7,1,5635,36.63,16.24,76.15
2,Camden,Audubon,401,5,1,7265,42.80,19.83,76.96
3,Camden,Audubon Park,402,5,1,852,54.45,16.73,76.17
4,Camden,Barrington,403,5,1,5495,42.59,19.99,74.58
5,Camden,Bellmawr,404,5,1,8741,46.92,17.36,71.94
6,Camden,Berlin,405,8,1,6163,32.93,26.34,75.01
7,Camden,Berlin Twp,406,6,1,4616,43.31,20.51,73.68
8,Camden,Brooklawn,407,5,1,1411,41.97,16.06,64.64
9,Camden,Camden,408,5,1,48023,53.28,4.55,45.71


Yay! Imported first data set for Congressional District 1 of New Jersey. Dataset includes number of registered voters by town, with overlapping variables for State-level legislative distrcts and Federal-level Congressional Districts. Final three variables include voting rates for Democrats, Republicans, and then total registered voters.

In [30]:
!wget -q -O Taxes.csv https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Taxes.csv

In [31]:
tax=pd.read_csv('Taxes.csv')

In [11]:
tax

,County,Municipality,DLGS Code,LD,CD,BudgetTwenty,MTRTwenty
0,Burlington,Maple Shade,319,6,1,1026,0.82
1,Burlington,Palmyra,327,7,1,1238,1.18
2,Camden,Audubon,401,5,1,1335,0.87
3,Camden,Audubon Park,402,5,1,1109,3.72
4,Camden,Barrington,403,5,1,1357,1.18
5,Camden,Bellmawr,404,5,1,1364,1.13
6,Camden,Berlin,405,8,1,1275,0.79
7,Camden,Berlin Twp,406,6,1,1901,1.20
8,Camden,Brooklawn,407,5,1,1980,1.68
9,Camden,Camden,408,5,1,2519,1.55


This second dataset includes similar data to the first dataset, but includes different variables. This dataset includes municipal-level budget data and municipal tax rate data. Both data sets were produced by the New Jersey Data Book. Initially, I had tried to use a different dataset as my second dataset, from the NJ Department of Community Affairs, which includes instances of planning permits being issued in excess of $250,000. For the purposes of PS2, I ditched that effort for two main reasons. First, the Community Affairs Dataset is huge, and in XLSX form. Second, it boiled down to time, and keeping things simple while learning. I met with Disability Services office this morning, and hope to have an accomodations letter sent to you soon.

Anyway, I do wish to return to the Community Affairs dataset because I think that will be a critical dataset for my dissertation for my analysis on actions taken by planning boards. I'd like to merge that dataset with data from the New Jersey Data Book to see if there is any correlation between big permits /$250K+/ and political party. My hypothesis is that there is not, but rather that there is a higher frequency in larger permits being issued in counties which run elections via a county-line bracketing system. Probably more to come!

In [32]:
tax.dtypes

County           object
Municipality     object
DLGS Code         int64
LD                int64
CD                int64
BudgetTwenty      int64
MTRTwenty       float64
dtype: object

Check the variables for which type of data they are. The types should match for the purposes of this dataset.

In [33]:
tax['LDST'] = tax.LD.astype(object) #tostring aok: yeah but in general turning storage of a number into a string doesnt make sense--why would you do that?

In [34]:
tax.dtypes

County           object
Municipality     object
DLGS Code         int64
LD                int64
CD                int64
BudgetTwenty      int64
MTRTwenty       float64
LDST             object
dtype: object

In [35]:
tax['BudgetTwentyST'] = tax.BudgetTwenty.astype(object) #tostring

In [ ]:
tax.dtypes

County             object
Municipality       object
DLGS Code           int64
LD                  int64
CD                  int64
BudgetTwenty        int64
MTRTwenty         float64
LDST               object
BudgetTwentyST     object
dtype: object

In [36]:
tax

,County,Municipality,DLGS Code,LD,CD,BudgetTwenty,MTRTwenty,LDST,BudgetTwentyST
0,Burlington,Maple Shade,319,6,1,1026,0.82,6,1026
1,Burlington,Palmyra,327,7,1,1238,1.18,7,1238
2,Camden,Audubon,401,5,1,1335,0.87,5,1335
3,Camden,Audubon Park,402,5,1,1109,3.72,5,1109
4,Camden,Barrington,403,5,1,1357,1.18,5,1357
5,Camden,Bellmawr,404,5,1,1364,1.13,5,1364
6,Camden,Berlin,405,8,1,1275,0.79,8,1275
7,Camden,Berlin Twp,406,6,1,1901,1.20,6,1901
8,Camden,Brooklawn,407,5,1,1980,1.68,5,1980
9,Camden,Camden,408,5,1,2519,1.55,5,2519


New dataset two variables recoded to string variables

In [37]:
tax = pd.DataFrame({"Municipality": ["Maple Shade", "Cherry Hill", "Logan", "Somerdale", "Pennsauken"],
                             "LDST": [3, 4, 5, 6, 7]})

Start of mapping

In [38]:
tax

,Municipality,LDST
0,Maple Shade,3
1,Cherry Hill,4
2,Logan,5
3,Somerdale,6
4,Pennsauken,7


In [40]:
LD_to_Party = {
  3: "Rep",
  4: "Dem",
  5: "Dem",
  6: "Dem",
  7: "Dem"}
tax["Party"] = tax["LDST"].map(LD_to_Party) #aok: now it works
tax

,Municipality,LDST,Party
0,Maple Shade,3,Rep
1,Cherry Hill,4,Dem
2,Logan,5,Dem
3,Somerdale,6,Dem
4,Pennsauken,7,Dem


Here, I am trying to map legislative district to the political party that currently represents it/has represented it as of 2020, year of the data set. I'm not sure why Dem and Rep come through as NaN...

In [ ]:
tax['colOfOnes'] = 1
tax['colOfOnes'][0:51]

0    1
1    1
2    1
3    1
4    1
Name: colOfOnes, dtype: int64

Column of Ones in order to begin groupby and agg commands

In [ ]:
tax.loc[tax['Party'] =='LDST', 'colOfOnes']  = '100' #but why would you do that? i did col of ones as a super basic example; its not useful for anything else
tax[['Party','colOfOnes']][0:51]

,Party,colOfOnes
0,NaN,1
1,NaN,1
2,NaN,1
3,NaN,1
4,NaN,1


In [42]:
tax[['LDST','Party', 'Municipality']].groupby('Party').agg(['mean', 'count']) #aok group by groups ie repetetive vals of var eg male/female! dont group by var with unique vals

<ipython-input-42-b7cc23f56792>:1: FutureWarning: ['Municipality'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  tax[['LDST','Party', 'Municipality']].groupby('Party').agg(['mean', 'count'])


LDST      
      mean count
Party           
Dem    5.5     4
Rep    3.0     1

Eventhough the party variable does not seem to be working correctly, I think I still executed this command correctly with the appropriately corresponding variables.

In [43]:
set(tax['County']).difference(set(perm['County'])) #.intersection(set(d1['id'])))

KeyError: ignored

Trying to execute the merge of both these datasets.

In [44]:
mergedOuter = pd.merge(tax, perm, how='outer', on=['County'],indicator=True) #outer=keep all #aok this didnt work!!! you dont have perm!
mergedOuter

KeyError: ignored